# Clean PyTorch customizable pipeline [Inference]

It's an inference part of "Fast&Automated clean PyTorch pipeline".   
Check out the training part [here](https://www.kaggle.com/vadimtimakin/fast-automated-clean-pytorch-pipeline-train).

You can use this inference for any PyTorch model. Just set its name in the config.

### Imports

In [ ]:
import torch
import torchvision.models as models
import numpy as np
import pandas as pd
import random
import os
import cv2
import torch.nn as nn
import albumentations as A
from albumentations.pytorch import ToTensor
from tqdm.notebook import tqdm

### Config

In [ ]:
class cfg:
    """Main config."""
    
    NUMCLASSES = 5  # CONST
    seed = 42       # random seed
    
    pathtoimgs = "../input/cassava-leaf-disease-classification/test_images"  # Path to folder with train images
    pathtocsv = "../input/cassava-leaf-disease-classification/sample_submission.csv"  # Path to csv-file with targets
    chk = "../input/cassava/weights.pt"  # Path to model checkpoint (weights)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Device
    modelname = "resnet18"  # PyTorch model
    
    apex = True            # Set True if you've used Apex for training
    apexoptlvl = "O1"      # Apex optimization level you've used for training
    batchsize = 1   # BatchSize
    numworkers = 4  # Number of workers
    
    # Transforms
    transforms = [
        dict(
            name="Resize",
            params=dict(
                height=256,
                width=256,
                p=1.0,
            )
        ),
        dict(
            name="Normalize",
            params=dict(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            )
        ),
        dict(
            name="/custom/totensor",
            params=dict(
            )
        ),
    ]

In [ ]:
print(cfg.device)

In [ ]:
if cfg.apex:    
    !git clone https://github.com/NVIDIA/apex && cd apex && pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" . --user && cd .. && rm -rf apex
    from apex import amp

### Custom transforms (check the training notebook)

In [ ]:
def totensor():
    return A.pytorch.ToTensor()

### Reproducibility

In [ ]:
def fullseed(seed=42):
    """Sets the random seeds."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    os.environ['PYTHONHASHSEED'] = str(seed)

    
fullseed(cfg.seed)

### Functions

In [ ]:
def get_model(cfg):
    """Get PyTorch model."""
    model = getattr(models, cfg.modelname)(pretrained=False)
    lastlayer = list(model._modules)[-1]
    setattr(model, lastlayer, nn.Linear(in_features=getattr(model, lastlayer).in_features,
                                        out_features=cfg.NUMCLASSES, bias=True))
    cp = torch.load(cfg.chk)
    if 'model' in cp:
        model.load_state_dict(cp['model'])
    else:
        model.load_state_dict(cp)
    if "amp" in cp and "optimizer" in cp:
        optimizer = cp["optimizer"]
        model, optimizer = amp.initialize(model, optimizer, opt_level=cfg.apexoptlvl, verbosity=0)
        amp.load_state_dict(cp['amp'])
    return model.to(cfg.device)


def get_transforms(cfg):
    """Get train and test augmentations."""
    transforms = [globals()[item["name"][8:]](**item["params"]) if item["name"].startswith("/custom/") 
                     else getattr(A, item["name"])(**item["params"]) for item in cfg.transforms]
    return A.Compose(transforms)

In [ ]:
class CassavaDataset(torch.utils.data.Dataset):
    """Cassava Dataset for uploading images and targets."""
    
    def __init__(self, cfg, images, transforms):
        self.images = images           # List with images
        self.transforms = transforms   # Transforms
        self.cfg = cfg                 # Config
        
    def __getitem__(self, idx):
        img = cv2.imread(os.path.join(self.cfg.pathtoimgs, self.images[idx]))
        img = torch.FloatTensor(self.transforms(image=img)["image"])
        return img

    def __len__(self):
        return len(self.images)

In [ ]:
def get_loader(cfg):
    """Getting dataloaders for train, validation (and test, if needed)."""
    data = pd.read_csv(cfg.pathtocsv)
    imgs = list(data["image_id"])
    transforms = get_transforms(cfg)
    dataset = CassavaDataset(cfg, imgs, transforms)
    dataloader = torch.utils.data.DataLoader(dataset,
                                             shuffle=False,
                                             batch_size=cfg.batchsize,
                                             pin_memory=True,
                                             num_workers=cfg.numworkers)
    return dataloader

## Inference

In [ ]:
torch.cuda.empty_cache()
dataloader = get_loader(cfg)
model = get_model(cfg) 
model.eval()

preds = []
with torch.no_grad():
    for img in tqdm(dataloader):
        outputs = model(img.to(cfg.device))
        preds.append(np.argmax(outputs.to('cpu').numpy()).tolist())

### Creating submission file

In [ ]:
print(preds)

In [ ]:
df = pd.read_csv(cfg.pathtocsv)
df.head()
df["label"] = preds
df.to_csv('submission.csv', index=False)
df.head()